In [ ]:
import numpy as np
import pandas as pd
import librosa
import os

In [ ]:
data_path = "C:\\Users\\user\\Downloads\\train"
set_a=pd.read_csv("train.csv")

In [ ]:
set_a.head()

,fname,label
0,healthy00001.wav,healthy
1,healthy00002.wav,healthy
2,healthy00003.wav,healthy
3,healthy00004.wav,healthy
4,healthy00005.wav,healthy


In [ ]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features

In [ ]:
from tqdm import tqdm
### Now we iterate through every audio file and extract features
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(set_a.iterrows()):
    file_name = os.path.join(os.path.abspath(data_path), str(row["fname"]))
    final_class_labels=row["label"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

2964it [19:15,  2.57it/s]


In [ ]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','label'])
extracted_features_df.head()

,feature,label
0,"[-464.35178, 42.00815, 35.443245, 27.206812, 1...",healthy
1,"[-537.3792, 100.824684, 79.680916, 53.019573, ...",healthy
2,"[-473.14993, 115.7308, 88.082375, 54.047775, 2...",healthy
3,"[-463.59232, 112.05316, 87.31859, 56.56482, 29...",healthy
4,"[-483.0581, 115.37701, 82.38545, 42.255928, 9....",healthy


In [ ]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['label'].tolist())

In [ ]:
X.shape

(2964, 40)

In [ ]:
y.shape

(2964,)

In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=2022)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
logi_reg=LogisticRegression()
logi_reg.fit(X_train,y_train)
logi_pred=logi_reg.predict(X_test)
acc_logi=accuracy_score(y_test, logi_pred)
print(acc_logi*100)

88.1956155143339


C:\Users\user\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.ensemble import RandomForestClassifier
mod_RF=RandomForestClassifier()
mod_RF.fit(X_train,y_train)
pred_RF=mod_RF.predict(X_test)
acc_RF=accuracy_score(y_test,pred_RF)
print(acc_RF*100)

94.09780775716695


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
mod_ada=AdaBoostClassifier()
mod_ada.fit(X_train,y_train)
pred_ada=mod_ada.predict(X_test)
acc_ada=accuracy_score(y_test, pred_ada)
print(acc_ada*100)

91.90556492411467


In [ ]:
from sklearn.svm import SVC
mod_svc=SVC(kernel='linear',probability = True)
mod_svc.fit(X_train,y_train)
pred_svc=mod_svc.predict(X_test)
acc_svc=accuracy_score(y_test,pred_svc)
print(acc_svc*100)

91.73693086003372


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
mod_tree = DecisionTreeClassifier()
mod_k = KNeighborsClassifier(n_neighbors=6)
mod_tree.fit(X_train, y_train)
mod_k.fit(X_train, y_train)
k_pred = mod_k.predict(X_test)
tree_pred = mod_tree.predict(X_test)

In [ ]:
acc_tree = accuracy_score(y_test, tree_pred)
acc_tree*100

91.2310286677909

In [ ]:
acc_knn = accuracy_score(y_test, k_pred)
acc_knn*100

91.56829679595279

In [ ]:
ACC=pd.DataFrame({'Algorithm':['Logistic Regression', 'Random Forest','Adaboost','SVM', 'Decision Tree Classifier', 'KNN'],'Accuracy':[acc_logi*100,acc_RF*100,acc_ada*100,acc_svc*100, acc_tree*100, acc_knn*100]},columns=['Algorithm','Accuracy'])

In [ ]:
ACC

,Algorithm,Accuracy
0,Logistic Regression,88.195616
1,Random Forest,94.097808
2,Adaboost,91.905565
3,SVM,91.736931
4,Decision Tree Classifier,91.231029
5,KNN,91.568297


In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
estimator_list = [
    ('knn',mod_k),
    ('svm',mod_svc),
    ('dtree',mod_tree),
    ('rf',mod_RF),
    ("ada",mod_ada)
]
stack_model = StackingClassifier(
    estimators=estimator_list, final_estimator=LogisticRegression())
# Train stacked model
stack_model.fit(X_train, y_train)

# Make predictions
y_train_pred = stack_model.predict(X_train)
y_test_pred = stack_model.predict(X_test)

# Training set model performance
stack_model_train_accuracy = accuracy_score(y_train, y_train_pred) # Calculate Accuracy
stack_model_train_mcc = matthews_corrcoef(y_train, y_train_pred) # Calculate MCC
stack_model_train_f1 = f1_score(y_train, y_train_pred, average='weighted') # Calculate F1-score

# Test set model performance
stack_model_test_accuracy = accuracy_score(y_test, y_test_pred) # Calculate Accuracy
stack_model_test_mcc = matthews_corrcoef(y_test, y_test_pred) # Calculate MCC
stack_model_test_f1 = f1_score(y_test, y_test_pred, average='weighted') # Calculate F1-score

print('Model performance for Training set')
print('- Accuracy: %s' % stack_model_train_accuracy)
print('- MCC: %s' % stack_model_train_mcc)
print('- F1 score: %s' % stack_model_train_f1)
print('----------------------------------')
print('Model performance for Test set')
print('- Accuracy: %s' % stack_model_test_accuracy)
print('- MCC: %s' % stack_model_test_mcc)
print('- F1 score: %s' % stack_model_test_f1)

Model performance for Training set
- Accuracy: 0.9949388443694643
- MCC: 0.9821334639785817
- F1 score: 0.9949588183162906
----------------------------------
Model performance for Test set
- Accuracy: 0.9460370994940978
- MCC: 0.8140679998429007
- F1 score: 0.9441263749621783


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_pred)

array([[476,   7],
       [ 25,  85]], dtype=int64)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

     healthy       0.95      0.99      0.97       483
   unhealthy       0.92      0.77      0.84       110

    accuracy                           0.95       593
   macro avg       0.94      0.88      0.90       593
weighted avg       0.95      0.95      0.94       593



In [ ]:
#DEEP

In [ ]:
### Label Encoding
#from tensorflow.keras.utils import to_categorical
#from sklearn.preprocessing import LabelEncoder
#labelencoder=LabelEncoder()
#y=to_categorical(labelencoder.fit_transform(y))
y=np.array(pd.get_dummies(y))


In [ ]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=2022)

In [ ]:
y.shape

(2964, 2)

In [ ]:
X_train.shape

(2371, 40)

In [ ]:
X_test.shape

(593, 40)

In [ ]:
y_train.shape

(2371, 2)

In [ ]:
y_test.shape

(593, 2)

In [ ]:
X_train

array([[-5.20704041e+02,  1.23896202e+02,  9.05382538e+01, ...,
        -1.60558140e+00, -2.68588603e-01,  7.51897991e-01],
       [-6.35821472e+02,  1.28705429e+02,  9.39685440e+01, ...,
         1.54518116e+00,  2.77337527e+00,  2.63617563e+00],
       [-4.92968201e+02,  1.65156815e+02,  1.18307076e+02, ...,
         2.47329974e+00,  3.15899014e+00,  2.24857044e+00],
       ...,
       [-4.95041931e+02,  8.05405121e+01,  6.50043106e+01, ...,
        -3.58712584e-01, -3.92459363e-01,  5.79564311e-02],
       [-4.26509094e+02,  1.25598045e+02,  8.93572769e+01, ...,
         1.47553325e+00,  2.53331661e+00,  2.25540209e+00],
       [-4.24604156e+02,  1.30097977e+02,  9.23661499e+01, ...,
         8.74363363e-01,  1.86571050e+00,  1.52698898e+00]], dtype=float32)

In [ ]:
y

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [0, 1],
       [0, 1],
       [0, 1]], dtype=uint8)

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [ ]:
### No of classes
num_labels=y.shape[1]

In [ ]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [ ]:
model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/heart_sound_classification.hdf5',
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/200
75/75 [==============================] - ETA: 0s - loss: 8.3848 - accuracy: 0.7592
Epoch 1: val_loss improved from inf to 0.82946, saving model to saved_models\heart_sound_classification.hdf5
75/75 [==============================] - 4s 16ms/step - loss: 8.3848 - accuracy: 0.7592 - val_loss: 0.8295 - val_accuracy: 0.8145
Epoch 2/200
11/75 [===>..........................] - ETA: 0s - loss: 3.5921 - accuracy: 0.7670

C:\Users\user\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


67/75 [=========================>....] - ETA: 0s - loss: 2.6259 - accuracy: 0.7579
Epoch 2: val_loss improved from 0.82946 to 0.61890, saving model to saved_models\heart_sound_classification.hdf5
75/75 [==============================] - 1s 9ms/step - loss: 2.5465 - accuracy: 0.7588 - val_loss: 0.6189 - val_accuracy: 0.8145
Epoch 3/200
71/75 [===========================>..] - ETA: 0s - loss: 1.3140 - accuracy: 0.7619
Epoch 3: val_loss did not improve from 0.61890
75/75 [==============================] - 1s 8ms/step - loss: 1.2858 - accuracy: 0.7647 - val_loss: 0.6685 - val_accuracy: 0.8145
Epoch 4/200
67/75 [=========================>....] - ETA: 0s - loss: 0.8591 - accuracy: 0.7757
Epoch 4: val_loss did not improve from 0.61890
75/75 [==============================] - 1s 7ms/step - loss: 0.8348 - accuracy: 0.7807 - val_loss: 0.6435 - val_accuracy: 0.8145
Epoch 5/200
71/75 [===========================>..] - ETA: 0s - loss: 0.6653 - accuracy: 0.7984
Epoch 5: val_loss improved from 0.6189

75/75 [==============================] - ETA: 0s - loss: 0.3434 - accuracy: 0.8317
Epoch 28: val_loss did not improve from 0.30938
75/75 [==============================] - 0s 6ms/step - loss: 0.3434 - accuracy: 0.8317 - val_loss: 0.3110 - val_accuracy: 0.8145
Epoch 29/200
71/75 [===========================>..] - ETA: 0s - loss: 0.3274 - accuracy: 0.8354
Epoch 29: val_loss improved from 0.30938 to 0.30518, saving model to saved_models\heart_sound_classification.hdf5
75/75 [==============================] - 0s 6ms/step - loss: 0.3277 - accuracy: 0.8334 - val_loss: 0.3052 - val_accuracy: 0.8145
Epoch 30/200
73/75 [============================>.] - ETA: 0s - loss: 0.3258 - accuracy: 0.8322
Epoch 30: val_loss improved from 0.30518 to 0.28956, saving model to saved_models\heart_sound_classification.hdf5
75/75 [==============================] - 0s 6ms/step - loss: 0.3261 - accuracy: 0.8317 - val_loss: 0.2896 - val_accuracy: 0.8145
Epoch 31/200
73/75 [============================>.] - ETA: 0s 

75/75 [==============================] - 0s 6ms/step - loss: 0.2769 - accuracy: 0.8705 - val_loss: 0.2455 - val_accuracy: 0.8786
Epoch 56/200
72/75 [===========================>..] - ETA: 0s - loss: 0.2817 - accuracy: 0.8663
Epoch 56: val_loss did not improve from 0.21567
75/75 [==============================] - 0s 6ms/step - loss: 0.2811 - accuracy: 0.8684 - val_loss: 0.2255 - val_accuracy: 0.9275
Epoch 57/200
70/75 [===========================>..] - ETA: 0s - loss: 0.2799 - accuracy: 0.8763
Epoch 57: val_loss did not improve from 0.21567
75/75 [==============================] - 0s 6ms/step - loss: 0.2744 - accuracy: 0.8790 - val_loss: 0.2199 - val_accuracy: 0.9089
Epoch 58/200
71/75 [===========================>..] - ETA: 0s - loss: 0.2681 - accuracy: 0.8798
Epoch 58: val_loss did not improve from 0.21567
75/75 [==============================] - 0s 6ms/step - loss: 0.2666 - accuracy: 0.8802 - val_loss: 0.2179 - val_accuracy: 0.9056
Epoch 59/200
69/75 [==========================>...] 

71/75 [===========================>..] - ETA: 0s - loss: 0.2430 - accuracy: 0.9049
Epoch 85: val_loss did not improve from 0.20589
75/75 [==============================] - 0s 6ms/step - loss: 0.2450 - accuracy: 0.9034 - val_loss: 0.2145 - val_accuracy: 0.9342
Epoch 86/200
69/75 [==========================>...] - ETA: 0s - loss: 0.2381 - accuracy: 0.8967
Epoch 86: val_loss did not improve from 0.20589
75/75 [==============================] - 0s 6ms/step - loss: 0.2369 - accuracy: 0.8971 - val_loss: 0.2242 - val_accuracy: 0.9157
Epoch 87/200
67/75 [=========================>....] - ETA: 0s - loss: 0.2784 - accuracy: 0.8797
Epoch 87: val_loss did not improve from 0.20589
75/75 [==============================] - 0s 6ms/step - loss: 0.2756 - accuracy: 0.8819 - val_loss: 0.2297 - val_accuracy: 0.9191
Epoch 88/200
75/75 [==============================] - ETA: 0s - loss: 0.2429 - accuracy: 0.8933
Epoch 88: val_loss improved from 0.20589 to 0.20340, saving model to saved_models\heart_sound_clas

Epoch 114/200
71/75 [===========================>..] - ETA: 0s - loss: 0.2273 - accuracy: 0.9071
Epoch 114: val_loss did not improve from 0.18248
75/75 [==============================] - 0s 6ms/step - loss: 0.2283 - accuracy: 0.9059 - val_loss: 0.1964 - val_accuracy: 0.9376
Epoch 115/200
71/75 [===========================>..] - ETA: 0s - loss: 0.2359 - accuracy: 0.9089
Epoch 115: val_loss did not improve from 0.18248
75/75 [==============================] - 0s 6ms/step - loss: 0.2368 - accuracy: 0.9089 - val_loss: 0.1854 - val_accuracy: 0.9410
Epoch 116/200
72/75 [===========================>..] - ETA: 0s - loss: 0.2399 - accuracy: 0.9023
Epoch 116: val_loss did not improve from 0.18248
75/75 [==============================] - 0s 6ms/step - loss: 0.2405 - accuracy: 0.9026 - val_loss: 0.2076 - val_accuracy: 0.9393
Epoch 117/200
71/75 [===========================>..] - ETA: 0s - loss: 0.2373 - accuracy: 0.9040
Epoch 117: val_loss did not improve from 0.18248
75/75 [======================

Epoch 144/200
68/75 [==========================>...] - ETA: 0s - loss: 0.2282 - accuracy: 0.9136
Epoch 144: val_loss did not improve from 0.18248
75/75 [==============================] - 0s 6ms/step - loss: 0.2284 - accuracy: 0.9119 - val_loss: 0.1968 - val_accuracy: 0.9410
Epoch 145/200
72/75 [===========================>..] - ETA: 0s - loss: 0.2296 - accuracy: 0.9084
Epoch 145: val_loss did not improve from 0.18248
75/75 [==============================] - 0s 6ms/step - loss: 0.2314 - accuracy: 0.9068 - val_loss: 0.2003 - val_accuracy: 0.9258
Epoch 146/200
71/75 [===========================>..] - ETA: 0s - loss: 0.2299 - accuracy: 0.9067
Epoch 146: val_loss did not improve from 0.18248
75/75 [==============================] - 0s 6ms/step - loss: 0.2276 - accuracy: 0.9076 - val_loss: 0.1895 - val_accuracy: 0.9393
Epoch 147/200
73/75 [============================>.] - ETA: 0s - loss: 0.2323 - accuracy: 0.9024
Epoch 147: val_loss did not improve from 0.18248
75/75 [======================

Epoch 174/200
65/75 [=========================>....] - ETA: 0s - loss: 0.2054 - accuracy: 0.9130
Epoch 174: val_loss did not improve from 0.17324
75/75 [==============================] - 0s 6ms/step - loss: 0.2055 - accuracy: 0.9161 - val_loss: 0.1793 - val_accuracy: 0.9359
Epoch 175/200
68/75 [==========================>...] - ETA: 0s - loss: 0.2131 - accuracy: 0.9113
Epoch 175: val_loss did not improve from 0.17324
75/75 [==============================] - 0s 6ms/step - loss: 0.2141 - accuracy: 0.9097 - val_loss: 0.1888 - val_accuracy: 0.9376
Epoch 176/200
68/75 [==========================>...] - ETA: 0s - loss: 0.2289 - accuracy: 0.8994
Epoch 176: val_loss did not improve from 0.17324
75/75 [==============================] - 0s 6ms/step - loss: 0.2280 - accuracy: 0.8992 - val_loss: 0.2005 - val_accuracy: 0.9292
Epoch 177/200
69/75 [==========================>...] - ETA: 0s - loss: 0.2239 - accuracy: 0.9049
Epoch 177: val_loss did not improve from 0.17324
75/75 [======================

In [ ]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.9409780502319336
